# CAR BUSINESS

## In this problem, we will be representing the role of a car salesman.

We will be taking a look at several conditions ownerships has told the salesman in order to satisfy the 2020 sales. There will be some constraints regarding the number of cars sold per model, as well as a minimum of vehicles that must be sold. The company at hand will be $BMW$, as it has many different options to choose from. 

The objective of this problem is to maximize profit selling the optimal numer of vehicles. 

_(Note: the optimal number of vehicles sold does not necessarily have to be equal to the minimum, because when purchasing a certain car, maintenance cost can also generate revenue for the company)_

![car](car.jpg)

The problem begins with loading the data, which is located in an $excel$ file which has 6 columns: 

$Price$: The cost of each vehicle.

$Model$: The name of the model of each car. (has categorical values)

$Total Units$: The number of cars available for each model.

$Production Cost$: The expenses the company has to pay.

$isElectric$: Parameter with two options, yes or no, depending on wether the car supports electric charge or not.

In [1]:
import pandas as pd

In [2]:
car = pd.read_excel('carData.xlsx')
print(car)

       Price        Model Category  Total Units  Production Cost isElectric
0    33950.0           X1      SUV           20          12901.0         No
1    46500.0           X3      SUV           20          17670.0         No
2    69200.0           X5      SUV           15          26296.0         No
3    71600.0           M3    Sedan           25          27208.0        Yes
4    74000.0           M4    Sedan           25          28120.0        Yes
5   103695.0          M5     Sedan           20          39404.1         No
6    39300.0           I3    Sedan           40          14934.0         No
7    39500.0      Series2    Coupé           45          15010.0         No
8    46000.0      Series3    Coupé           45          17480.0         No
9    56600.0      Series5    Coupé           30          21508.0         No
10   62260.0   Series5_GT    Coupé           10          23658.8         No
11  136600.0  i8_Roadster    Sport            5          51908.0         No
12   58000.0

In [3]:

# Start the model and intilialize the variables and parameters.

price = car['Price']
model_car = car['Model']
total_units = car['Total Units']
production_cost = car['Production Cost']
electric = car['isElectric']
category = car['Category']

In [4]:
from pyomo.environ import *

model = ConcreteModel()

# we introduce the model, and initialize the set and the parameters.

model.model_car = Set(initialize=['X1', 'X3', 'X5', 'M3', 'M4', 'M5', 'I3', 'Series2', 'Series3', 'Series5', 
                                 'Series5_GT', 'i8_Roadster', 'z4_Roadster', 'alpina_B8', 'x3_M'], doc='model_car' )


model.category = Param(model.model_car, initialize = {'X1':category[0], 'X3':category[1], 'X5':category[2], 
                                                   'M3':category[3], 'M4':category[4], 'M5':category[5], 
                                                   'I3':category[6], 'Series2':category[7], 'Series3':category[8], 
                                                   'Series5':category[9], 'Series5_GT':category[10], 
                                                   'i8_Roadster':category[11],'z4_Roadster': category[12], 
                                                   'alpina_B8':category[13], 'x3_M':price[14]}, doc = 'Category for each car',
                              within = Any)



model.electric = Param(model.model_car, initialize = {'X1':electric[0], 'X3':electric[1], 'X5':electric[2], 
                                                   'M3':electric[3], 'M4':electric[4], 'M5':electric[5], 
                                                   'I3':electric[6], 'Series2':electric[7], 'Series3':electric[8], 
                                                   'Series5':electric[9], 'Series5_GT':electric[10], 
                                                   'i8_Roadster':electric[11], 'z4_Roadster': electric[12], 
                                                   'alpina_B8':electric[13], 'x3_M':electric[14]}, 
                               doc = 'Wether a car is electric or not', within = Any)


model.price = Param(model.model_car, initialize = {'X1':price[0], 'X3':price[1], 'X5':price[2], 
                                                   'M3':price[3], 'M4':price[4], 'M5':price[5], 
                                                   'I3':price[6], 'Series2':price[7], 'Series3':price[8], 
                                                   'Series5':price[9], 'Series5_GT':price[10], 
                                                   'i8_Roadster':price[11],'z4_Roadster': price[12], 
                                                   'alpina_B8':price[13], 'x3_M':price[14]}, doc = 'Price for each car')


model.production_cost = Param(model.model_car, initialize = {'X1':production_cost[0], 'X3':production_cost[1], 
                                                             'X5':production_cost[2], 'M3':production_cost[3], 
                                                             'M4':production_cost[4], 'M5':production_cost[5], 
                                                             'I3':production_cost[6], 'Series2':production_cost[7], 
                                                             'Series3':production_cost[8], 'Series5':production_cost[9], 
                                                             'Series5_GT':production_cost[10], 'i8_Roadster':production_cost[11],
                                                            'z4_Roadster':production_cost[12], 'alpina_B8':production_cost[13], 
                                                             'x3_M':production_cost[14]}, 
                    doc = 'Production cost for each car')


model.units = Param(model.model_car, initialize = {'X1':total_units[0], 'X3':total_units[1], 'X5':total_units[2], 
                                                   'M3':total_units[3], 'M4':total_units[4], 'M5':total_units[5], 
                                                   'I3':total_units[6], 'Series2':total_units[7], 'Series3':total_units[8], 
                                                   'Series5':total_units[9], 'Series5_GT':total_units[10], 
                                                   'i8_Roadster':total_units[11], 'z4_Roadster': total_units[12], 
                                                   'alpina_B8':total_units[13], 'x3_M':total_units[14]}, doc = 'Number of units for each car', 
                    within = NonNegativeIntegers)

model.max_customers = Param(initialize= 100, doc='desired number of customers')

model.min_customers = Param(initialize = 20, doc='minimum customers')

model.target_min = Param(initialize=1e6, doc='Minimum expense')

# This is the variable we will be working with.

model.units_sold = Var(model.model_car, doc='number of units sold', within=NonNegativeReals)

# we want to find the number of cars sold for each model, using several constraints


Having the important information of the model, we will move on to creating the constraints. There will be many different constraint that affect the solution in a different way.

The constraints are the following:

A maximum number of customers (100)

A minimum number of customers (20)

A target of at least one million dollars in revenue.
   
For every car model, there is a maximum number of units (another constraint)

We want at least 20 electric cars, exactly 15 SUVs and less than 10 sport cars.
    

In [5]:
# Objective function: we want to maximize profit
def objective_rule(model):
    return sum(model.units_sold[i]*model.price[i] for i in model.model_car) - sum(model.units_sold[i]*model.production_cost[i] 
                                                                                  for i in model.model_car)
model.objective = Objective(rule=objective_rule, sense=maximize, doc='maximize the profit')

# Constraints
# the maximum number of customers
def max_customers_cons(model):
    return sum(model.units_sold[i] for i in model.model_car) <= model.max_customers
model.max_customers_cons = Constraint(rule=max_customers_cons, doc='customers constraint')
max

# we need a minimum of customers
def min_customers_cons(model):
    return sum(model.units_sold[i] for i in model.model_car) >= model.min_customers
model.min_customers_cons = Constraint(rule=min_customers_cons, doc='customers constraint')

# this condition takes into account only the retail price, not the earnings.That is, we want to sell at least one million worth
# of profit, without considering the costs for producing the cars.
def Target_min(model):
    return sum(model.units_sold[i]*model.price[i] for i in model.model_car) >= model.target_min
model.Target_min = Constraint(rule=Target_min, doc='Minimum target constraint')

# for every car model, there is a maximum of available units.
def max_number(model, index):
    return model.units_sold[index] <= model.units[index]
model.max_number = Constraint(model.model_car, rule=max_number, doc='maximum number of cars')

# now we will introduce some more constraints

# for the first one: at least 20% of cars have to be electric
def electric_20(model):
    return sum(model.units_sold[i] for i in model.model_car if model.electric[i] == 'Yes') >= 20
model.electric_20 = Constraint(rule=electric_20, doc='At least 20 cars must be electric')


# we aslo want exactly 15 SUV 
def suv_15(model):
    return sum(model.units_sold[i] for i in model.model_car if model.category[i] == 'SUV') == 15
model.suv_15 = Constraint(rule=suv_15, doc='There have to be exactly 15 SUVs')


# lastly we do not want more than 10 sport cars sold
def less_10_sport(model):
    return sum(model.units_sold[i] for i in model.model_car if model.category[i] == 'Sport') <= 10
model.less_10_sport = Constraint(rule=less_10_sport, doc='There have to be less than 10 Sport cars')




$c)$

In [6]:

model.dual = Suffix(direction=Suffix.IMPORT)#, datatype=Suffix.INT) 
Solver = SolverFactory('glpk')
Results = Solver.solve(model)


In [7]:
# we take a look at the solution

model.display()

Model unknown

  Variables:
    units_sold : number of units sold
        Size=15, Index=model_car
        Key         : Lower : Value            : Upper : Fixed : Stale : Domain
                 I3 :     0 :              0.0 :  None : False : False : NonNegativeReals
                 M3 :     0 :             25.0 :  None : False : False : NonNegativeReals
                 M4 :     0 :             25.0 :  None : False : False : NonNegativeReals
                 M5 :     0 :             20.0 :  None : False : False : NonNegativeReals
            Series2 :     0 :              0.0 :  None : False : False : NonNegativeReals
            Series3 :     0 :              0.0 :  None : False : False : NonNegativeReals
            Series5 :     0 :              0.0 :  None : False : False : NonNegativeReals
         Series5_GT :     0 : 5.00000000000001 :  None : False : False : NonNegativeReals
                 X1 :     0 :              0.0 :  None : False : False : NonNegativeReals
           

### Interpreting the solution:

First of all, we get an income of 5286244.0 dollars. The models that will be sold are the following: 25 $M3$, 25 $M4$, 20 $M5$, 5 $Series5GT$, 15 $X5$, 4 $alpinaB8$, 5 $i8Roadster$, and one $x3M$

In [8]:
model.dual.display()

dual : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key                     : Value
                 Target_min :     0.0
                electric_20 :     0.0
              less_10_sport :     0.0
         max_customers_cons : 38601.2
             max_number[I3] :     0.0
             max_number[M3] :  5790.8
             max_number[M4] :  7278.8
             max_number[M5] : 25689.7
        max_number[Series2] :     0.0
        max_number[Series3] :     0.0
        max_number[Series5] :     0.0
     max_number[Series5_GT] :     0.0
             max_number[X1] :     0.0
             max_number[X3] :     0.0
             max_number[X5] : 14074.0
      max_number[alpina_B8] : 51298.8
    max_number[i8_Roadster] : 46090.8
           max_number[x3_M] : 85398.8
    max_number[z4_Roadster] :     0.0
         min_customers_cons :     0.0
                     suv_15 : -9771.2


Taking a look at results, we can conclude that there are some variables that carry the most importance for the model. Those cars that sell the best are the ones limited by their respective constraints: If the maximum number of cars constraint was not taken into account, all the units of just three models would be sold: $x3M$, $X5$ and $M4$. However, when itroducing this last part, we can see that the maximum number of units heavily affects the types of cars sold. 
Of course, having a maximum number of clients is also important, so that constraint carries some weight. Lastly, making the constraint of having exactly 15 $SUVs$ was not really flexible for the model, as it made the user purchase some types of cars that might not have been sold, so that is why is has negative value


<img src="bmwSport.jpg" width="350">

# Binary Part

$d)$

Having obtained the aforementioned results, we will change the problem a little. Now, if we wanted to sell a model, we can do so, but there is one difference: we can either sell all existing units, or none. That way, we introduce binary variables to the formulation. 

Here we can take a look at the new model, with some new features, and more constraints. There is an additional column, $extra$ which says if the car has option for extra features or not. It will be used in one of the constraints.

In [9]:
carLogical = pd.read_excel ('carDataLogical.xlsx')
print(carLogical)

       Price        Model Category  Total Units Extras  Production Cost  \
0    33950.0           X1      SUV           20     No          12901.0   
1    46500.0           X3      SUV           20     No          17670.0   
2    69200.0           X5      SUV           15    Yes          26296.0   
3    71600.0           M3    Sedan           25    Yes          27208.0   
4    74000.0           M4    Sedan           25    Yes          28120.0   
5   103695.0          M5     Sedan           20     No          39404.1   
6    39300.0           I3    Sedan           40     No          14934.0   
7    39500.0      Series2    Coupé           45     No          15010.0   
8    46000.0      Series3    Coupé           45     No          17480.0   
9    56600.0      Series5    Coupé           30    Yes          21508.0   
10   62260.0   Series5_GT    Coupé           10    Yes          23658.8   
11  136600.0  i8_Roadster    Sport            5    Yes          51908.0   
12   58000.0  z4_Roadster

In [10]:
from pyomo.environ import *

# Start the model
model_logical = AbstractModel()
price = carLogical['Price']
model_car = carLogical['Model']
category = carLogical['Category']
total_units = carLogical['Total Units']
production_cost = carLogical['Production Cost']
extra = carLogical['Extras']
electric = carLogical['isElectric']


In [11]:
from pyomo.environ import *

# we initialize the variables of the model the same way. However, when creating the variable, it will be $binary$.

model_logical = ConcreteModel()

model_logical.model_car = Set(initialize=['X1', 'X3', 'X5', 'M3', 'M4', 'M5', 'I3', 'Series2', 'Series3', 'Series5', 
                                 'Series5_GT', 'i8_Roadster', 'z4_Roadster', 'alpina_B8', 'x3_M'], doc='model_car' )


model_logical.units = Param(model_logical.model_car, initialize = {'X1':total_units[0], 'X3':total_units[1], 'X5':total_units[2], 
                                                   'M3':total_units[3], 'M4':total_units[4], 'M5':total_units[5], 
                                                   'I3':total_units[6], 'Series2':total_units[7], 'Series3':total_units[8], 
                                                   'Series5':total_units[9], 'Series5_GT':total_units[10], 
                                                   'i8_Roadster':total_units[11], 'z4_Roadster': total_units[12], 
                                                   'alpina_B8':total_units[13], 'x3_M':total_units[14]}, doc = 'Number of units for each car', 
                    within = NonNegativeIntegers)


model_logical.extra = Param(model_logical.model_car, initialize = {'X1':extra[0], 'X3':extra[1], 'X5':extra[2], 
                                                   'M3':extra[3], 'M4':extra[4], 'M5':extra[5], 
                                                   'I3':extra[6], 'Series2':extra[7], 'Series3':extra[8], 
                                                   'Series5':extra[9], 'Series5_GT':extra[10], 
                                                   'i8_Roadster':extra[11], 'z4_Roadster': extra[12], 
                                                   'alpina_B8':extra[13], 'x3_M':extra[14]}, doc = 'If a car has extra features or not', 
                    within = Any)


model_logical.electric = Param(model_logical.model_car, initialize = {'X1':electric[0], 'X3':electric[1], 'X5':electric[2], 
                                                   'M3':electric[3], 'M4':electric[4], 'M5':electric[5], 
                                                   'I3':electric[6], 'Series2':electric[7], 'Series3':electric[8], 
                                                   'Series5':electric[9], 'Series5_GT':electric[10], 
                                                   'i8_Roadster':electric[11], 'z4_Roadster': electric[12], 
                                                   'alpina_B8':electric[13], 'x3_M':electric[14]}, 
                               doc = 'Number of units for each car', within = Any)



model_logical.category = Param(model_logical.model_car, initialize = {'X1':category[0], 'X3':category[1], 'X5':category[2], 
                                                   'M3':category[3], 'M4':category[4], 'M5':category[5], 
                                                   'I3':category[6], 'Series2':category[7], 'Series3':category[8], 
                                                   'Series5':category[9], 'Series5_GT':category[10], 
                                                   'i8_Roadster':category[11],'z4_Roadster': category[12], 
                                                   'alpina_B8':category[13], 'x3_M':price[14]}, doc = 'Category for each car',
                              within = Any)


model_logical.price = Param(model_logical.model_car, initialize = {'X1':price[0], 'X3':price[1], 'X5':price[2], 
                                                   'M3':price[3], 'M4':price[4], 'M5':price[5], 
                                                   'I3':price[6], 'Series2':price[7], 'Series3':price[8], 
                                                   'Series5':price[9], 'Series5_GT':price[10], 
                                                   'i8_Roadster':price[11],'z4_Roadster': price[12], 
                                                   'alpina_B8':price[13], 'x3_M':price[14]}, doc = 'Price for each car')


model_logical.production_cost = Param(model_logical.model_car, initialize = {'X1':production_cost[0], 'X3':production_cost[1], 
                                                             'X5':production_cost[2], 'M3':production_cost[3], 
                                                             'M4':production_cost[4], 'M5':production_cost[5], 
                                                             'I3':production_cost[6], 'Series2':production_cost[7], 
                                                             'Series3':production_cost[8], 'Series5':production_cost[9], 
                                                             'Series5_GT':production_cost[10], 'i8_Roadster':production_cost[11],
                                                            'z4_Roadster':production_cost[12], 'alpina_B8':production_cost[13], 
                                                             'x3_M':production_cost[14]}, 
                    doc = 'Production cost for each car')


model_logical.max_customers = Param(initialize= 100, doc='desired number of customers')

model_logical.min_customers = Param(initialize = 5, doc='minimum customers')

model_logical.target_min = Param(initialize=1e6, doc='Minimum expense')

# variable

model_logical.units_sold = Var(model_logical.model_car, doc='number of units sold', within=Binary)

# we want to find the number of cars sold for each model, using several constraints



In [12]:
# Objective function remains the same
def objective_rule(model_logical):
    return sum(model_logical.units_sold[i]*model_logical.price[i]*model_logical.units[i]
               for i in model_logical.model_car) - sum(
        model_logical.units_sold[i]*model_logical.production_cost[i]*model_logical.units[i] for i in model_logical.model_car)

model_logical.objective = Objective(rule=objective_rule, sense=maximize, doc='total ROI')

# Constraints
def max_customers_cons(model_logical):
    return sum(model_logical.units_sold[i]*model_logical.units[i] for i in model_logical.model_car) <= model_logical.max_customers
model_logical.max_customers_cons = Constraint(rule=max_customers_cons, doc='customers constraint')

def min_customers_cons(model_logical):
    return sum(model_logical.units_sold[i]*model_logical.units[i] for i in model_logical.model_car) >= model_logical.min_customers
model_logical.min_customers_cons = Constraint(rule=min_customers_cons, doc='customers constraint')

def Target_min(model_logical):
    return sum(model_logical.units_sold[i]*model_logical.price[i]*model_logical.units[i] 
               for i in model_logical.model_car)>= model_logical.target_min
model_logical.Target_min = Constraint(rule=Target_min, doc='Minimum target constraint')


def max_number(model_logical, index):
    return model_logical.units_sold[index]*model_logical.units[index] <= model_logical.units[index]
model_logical.max_number = Constraint(model_logical.model_car, rule=max_number, doc='maximum number of cars')



def electric_20(model_logical):
    return sum(model_logical.units_sold[i]*model_logical.units[i] 
               for i in model_logical.model_car if model_logical.electric[i] == 'Yes') >= 20
model_logical.electric_20 = Constraint(rule=electric_20, doc='At least 20 cars must be electric')


def suv_15(model_logical):
    return sum(model_logical.units_sold[i]*model_logical.units[i] 
               for i in model_logical.model_car if model_logical.category[i] == 'SUV') == 15
model_logical.suv_15 = Constraint(rule=suv_15, doc='There have to be exactly 15 SUVs')


def less_10_sport(model_logical):
    return sum(model_logical.units_sold[i]*model_logical.units[i] for i in model_logical.model_car if model_logical.category[i] == 'Sport') <= 10
model_logical.less_10_sport = Constraint(rule=less_10_sport, doc='There have to be less than 10 Sport cars')

# these are the same constraints we saw before
# our last contraint, that was not included in the previous model, will be regarding extra features on cars.
def extra_50(model_logical):
    return sum(model_logical.units_sold[i]*model_logical.units[i] 
               for i in model_logical.model_car if model_logical.extra[i] == 'No') <= 50
model_logical.extra_25 = Constraint(rule=extra_50, doc='The is a maximum of 50 sold cars with extra features')


In [13]:
# these are some binary constraints.

model_logical.cons_bin1 = Constraint(expr = model_logical.units_sold['X1'] + model_logical.units_sold['X3'] + 
                                    model_logical.units_sold['X5'] <= 1)

model_logical.cons_bin2 = Constraint(expr = model_logical.units_sold['z4_Roadster'] + model_logical.units_sold['alpina_B8'] <= 1)

model_logical.cons_bin3 = Constraint(expr = model_logical.units_sold['x3_M'] + model_logical.units_sold['i8_Roadster'] == 0)

model_logical.cons_bin4 = Constraint(expr = model_logical.units_sold['M4'] + model_logical.units_sold['M5'] == 0)



## How binary constraints affect the result


It is clear to see that when using these new constraints, the results are gona vary with respect to the old model. In this case, we cannot have $M4$ and $M5$ models simultaneously, we cannot have any $x3M$ or $i8Roadster$ either, as well as some other conditions. They were chosen to change the answer we obtained before.

In [14]:
Solver = SolverFactory('glpk')
Results = Solver.solve(model_logical)

In [15]:
model_logical.display()

Model unknown

  Variables:
    units_sold : number of units sold
        Size=15, Index=model_car
        Key         : Lower : Value : Upper : Fixed : Stale : Domain
                 I3 :     0 :   0.0 :     1 : False : False : Binary
                 M3 :     0 :   1.0 :     1 : False : False : Binary
                 M4 :     0 :   0.0 :     1 : False : False : Binary
                 M5 :     0 :   0.0 :     1 : False : False : Binary
            Series2 :     0 :   0.0 :     1 : False : False : Binary
            Series3 :     0 :   1.0 :     1 : False : False : Binary
            Series5 :     0 :   0.0 :     1 : False : False : Binary
         Series5_GT :     0 :   1.0 :     1 : False : False : Binary
                 X1 :     0 :   0.0 :     1 : False : False : Binary
                 X3 :     0 :   0.0 :     1 : False : False : Binary
                 X5 :     0 :   1.0 :     1 : False : False : Binary
          alpina_B8 :     0 :   1.0 :     1 : False : False : Binary
    

As we can see, the type of the constraints is now $binary$ for the last ones, which affects the problem in a considerable way. 
First of all, comparing the income we obtain, we can see that selling just a bunch of cars of the same model is not the ideal woy for the company to work. Before, as cars could be sold units-wise, the model was able to be much better, and obtain a profit of 5286244 dollars, meanwhile, using $binary$ conditions, we obtain just 3782372.

<img src="SUV.jpg" width="400">